In [ ]:
# Run this cell to install DiffeRT and its dependencies, e.g., on Google Colab

try:
    import differt  # noqa: F401
except ImportError:
    import sys  # noqa: F401

    !{sys.executable} -m pip install differt[all]

# Coherence Map

In [51]:
import equinox as eqx
import jax.numpy as jnp
import numpy as np

from differt.geometry.utils import path_lengths
from differt.plotting import draw_image, reuse, set_defaults
from differt.scene.sionna import download_sionna_scenes, get_sionna_scene
from differt.scene.triangle_scene import TriangleScene

In [52]:
download_sionna_scenes()  # Let's download Sionna scenes (from the main branch)

In [ ]:
set_defaults("plotly")

file = get_sionna_scene("simple_street_canyon")
scene = TriangleScene.load_xml(file)

scene = eqx.tree_at(lambda s: s.transmitters, scene, jnp.array([-33.0, 0.0, 32.0]))
batch = (50, 50)
scene_grid = scene.with_receivers_grid(*batch)
power = jnp.zeros(batch)
masks = jnp.zeros((*batch, 0), dtype=bool)

x, y, z = jnp.unstack(scene_grid.receivers, axis=-1)

with reuse() as fig:
    scene.plot()

    for order in range(2):
        paths = scene_grid.compute_paths(order=order)
        masks = jnp.concatenate((masks, paths.mask), axis=-1)
        power += (paths.mask / path_lengths(paths.vertices) ** 2).sum(axis=-1)

    draw_image(
        np.asarray(power), x=np.asarray(x), y=np.asarray(y), z0=float(z.ravel()[0])
    )

fig

In [ ]:
with reuse() as fig:
    scene.plot()
    unique = jnp.unique(
        masks.reshape(-1, masks.shape[-1]), axis=0, return_inverse=True
    )[1].reshape(masks.shape[:-1])
    draw_image(
        unique / unique.max(), x=np.asarray(x), y=np.asarray(y), z0=float(z.ravel()[0])
    )

fig